<a href="https://colab.research.google.com/github/mqquiroz/Fluid_Mechanics/blob/main/Ejercicios_Tuberias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal
import pandas as pd

#Método de Newton-Raphson
# Para encontrar el factor de friccion en funcion de e/D y Re
def f_friccion_NR(e_D,Re,tol=1e-6):
  x0 = 0.03
  while True:
    f = 1/np.sqrt(x0) + 2*np.log10((e_D/3.7)+(2.51/(Re*np.sqrt(x0))))
    fprima = -1/np.sqrt(x0**3) - ( (2.51/(Re*np.sqrt(x0**3)))/( (e_D/3.7) + (2.51/(Re*np.sqrt(x0))) )  )*np.log10(np.exp(1))
    dxx = -f/fprima
    if np.abs(dxx/x0) < tol:
      xx = x0
      break
    x0 = x0 + dxx
  return xx

def Area(D):
  A = (np.pi*D**2)/4
  return A

def Reynolds(U,D,nu):
  Re = U*D/nu
  return Re

def Reynolds2(rho,U,D,mu):
  Re = rho*U*D/mu
  return Re

#Constantes
g   = 9.81 #(m/s^2)
rho = 1000 #(kg/m^3)

## Ejercicios de Fluidos en Tuberías

### Ejercicio 1
¿Cuál debe ser el diámetro de una tubería de fierro forjado (rugosidad absoluta 0.05 mm)
para transportar 15 l/s de agua a 15 °C (viscosidad cinemática $1.12\cdot 10^{-6}$ m$^{2}$/s) con una
altura de disponible de 40 m? Considere sólo las pérdidas regulares

Balance de Energía (Bernoulli):
\begin{align*}
B_{1} &= B_{2} + \Delta h_{f}\\
z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} \\
z_{1} - z_{2} &= \Delta h_{f} \\
40 [m] &= f\frac{L}{D} \frac{U^{2}}{2g} = f\frac{L}{D} \frac{(Q/A)^{2}}{2g} = \frac{8fL}{g\pi^{2}D^{5}} Q^{2}
\end{align*}

A su vez:

$$Re = \dfrac{UD}{\nu}$$

$$ f = funcion\left(Re,\dfrac{\epsilon}{D}\right) $$

El problema debe ser resuelto de forma iterativa.

Procedimiento: \\

1) Suponer un valor de $f_{supuesto}$ \\
2) Utilizar $40 [m] = \frac{8f_{supuesto}L}{g\pi^{2}D^{5}} Q^{2}$ para resolver D \\
3) Calcular $U$ con $Q$ y $D$ \\
4) Calcular Número de Reynolds $Re =\dfrac{UD}{\nu}$ \\
5) Calcular rugosidad relativa $\dfrac{\epsilon}{D}$ \\
6) Recalcular $f$ del Diagrama de Moody

7) Si $f_{supuesto}=f$ entonces OK; sino suponer el valor de $f$ encontrado y reiniciar procedimiento.

In [2]:
def Ecuacion_D(f_sup,L,g,Q):
  term1 = 8*f_sup*L
  term2 = 40*g*(np.pi**2)
  term3 = (term1/term2)
  term4 = term3*(Q**2)
  D     = (term4)**(1/5)
  return D

# Datos
epsilon = 0.05 # Rugoisidad absoluta (mm)
Q       = 15 # Caudal (l/s)
nu      = 1.12*1e-6 # Viscosidad Cinemática (m^2/s)
H       = 40 #Altura de Energía Disponible (m)
L       = 200 # Largo de la Tubería (m)

#--------------------------------

Q = Q/1000 #(m3/s)

tol=1e-3
M = np.array([[]])
f_sup = 0.03

i = 0
while True:
  D = Ecuacion_D(f_sup,L,g,Q)
  #print('D = ',np.round(D,3),' m')
  A = Area(D)
  #print('A = ',np.round(A,3),' m^2')
  U = Q/A
  #print('U = ',np.round(U,3),' m/s')
  Re = Reynolds(U,D,nu)
  #print('Re = ','%.2E' % Decimal(Re))
  e_D = epsilon/(D*1000)
  #print('epsilon/D = ','%.6f' % Decimal(e_D))
  f = f_friccion_NR(e_D,Re)
  #print('f = ','%.6f' % Decimal(f))

  #M.append([f_sup, D, A, Re, e_D, f])
  M = np.append(M, [int(i+1),f_sup, D, A, U, Re, e_D, f])
  if np.abs((f-f_sup)/f) < tol:
    break
  f_sup = f
  i = i + 1

M = M.reshape(int(len(M)/8),8)
# Convert the NumPy array to a DataFrame
df = pd.DataFrame(M, columns=['Iteración','f_Supuesto','D (m)','A (m^2)','U (m/s)','Re','e_D','f'])
df['Iteración'] = df['Iteración'].map('{:,.0f}'.format)
df['Re'] = df['Re'].map('{:,.2e}'.format)
df['D (m)'] = df['D (m)'].map('{:.3f}'.format)
df['A (m^2)'] = df['A (m^2)'].map('{:.3f}'.format)
df['U (m/s)'] = df['U (m/s)'].map('{:.3f}'.format)

# displaying the DataFrame
display(df)

,Iteración,f_Supuesto,D (m),A (m^2),U (m/s),Re,e_D,f
0,1,0.030000,0.077,0.005,3.183,2.20e+05,0.000645,0.019385
1,2,0.019385,0.071,0.004,3.791,2.40e+05,0.000704,0.019544
2,3,0.019544,0.071,0.004,3.778,2.40e+05,0.000703,0.019541


### Ejercicio 2
Calcular el caudal que circula por el sistema indicado en la figura despreciando todas las pérdidas excepto las que tienen lugar por fricción en la tubería.

Balance de Energía (Bernoulli):
\begin{align*}
B_{1} &= B_{2} + \Delta h_{f}\\
z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} + \Delta h_{f}\\
z_{1} - z_{2} &= \Delta h_{f} \\
6 [m] &= f\frac{L}{D} \frac{U^{2}}{2g}
\end{align*}

A su vez:

$$Re = \dfrac{UD}{\nu}$$

$$ f = funcion\left(Re,\dfrac{\epsilon}{D}\right) $$

El problema debe ser resuelto de forma iterativa.

Procedimiento: \\

1) Suponer un valor de $f_{supuesto}$ \\
2) Utilizar $6 [m] = f_{supuesto}\frac{L}{D} \frac{U^{2}}{2g} $ \\
3) Calcular $U$ \\
4) Calcular Número de Reynolds $Re =\dfrac{\rho U D}{\mu}$ \\
5) Calcular rugosidad relativa $\dfrac{\epsilon}{D}$ \\
6) Recalcular $f$ del Diagrama de Moody

7) Si $f_{supuesto}=f$ entonces OK; sino suponer el valor de $f$ encontrado y reiniciar procedimiento.

In [3]:
def Ecuacion_U(f_sup,L,g,D,H):
  term1 = 2*g*H*D
  term2 = f_sup*L
  term3 = (term1/term2)
  U = np.sqrt(term3)
  return U

# Datos
epsilon = 0.06 # Rugoisidad absoluta (mm)
D       = 60 # Diámetro (mm)
mu      = 0.1 # Viscosidad Dinámica (absoluta) (poise)
H       = 6 #Altura de Energía Disponible (m)
L       = 4.8 # Largo de la Tubería (m)
rho     = 880 # Densidad (kg/m3)

#--------------------------------

D       = D/1000 #(m)
epsilon = epsilon/1000 #(m)
mu      = mu*0.1 # (Pa s)

e_D = epsilon/D
A   = Area(D)

tol=1e-3
M = np.array([[]])
f_sup = 0.03

i = 0
while True:
  U = Ecuacion_U(f_sup,L,g,D,H)
  #print('U = ',np.round(U,3),' m/s')
  Q = U*A
  #print('Q = ',np.round(Q,3),' m3/s')
  Re = Reynolds2(rho,U,D,mu)
  #print('Re = ','%.2E' % Decimal(Re))
  f = f_friccion_NR(e_D,Re)
  #print('f = ','%.6f' % Decimal(f))

  #M.append([f_sup, D, A, Re, e_D, f])
  M = np.append(M, [int(i+1),f_sup, U, Q, Re, f])
  if np.abs((f-f_sup)/f) < tol:
    break
  f_sup = f
  i = i + 1

M = M.reshape(int(len(M)/6),6)
# Convert the NumPy array to a DataFrame
df = pd.DataFrame(M, columns=['Iteración','f_Supuesto','U (m/s)','Q (m^3/s)','Re','f'])
df['Iteración'] = df['Iteración'].map('{:,.0f}'.format)
df['Re'] = df['Re'].map('{:,.2e}'.format)
df['U (m/s)'] = df['U (m/s)'].map('{:.3f}'.format)
df['Q (m^3/s)'] = df['Q (m^3/s)'].map('{:.3f}'.format)

# displaying the DataFrame
display(df)

,Iteración,f_Supuesto,U (m/s),Q (m^3/s),Re,f
0,1,0.030000,7.004,0.020,3.70e+04,0.025104
1,2,0.025104,7.656,0.022,4.04e+04,0.024764
2,3,0.024764,7.708,0.022,4.07e+04,0.024739
3,4,0.024739,7.712,0.022,4.07e+04,0.024737


### Ejercicio 3
Determinar el caudal que circula en el sistema de la figura y dibujar las líneas de energía.

Balance de Energía (Bernoulli):
\begin{align*}
B_{1} &= B_{2} + \Delta h_{f} + \Delta h_{s}\\
z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} + \Delta h_{f} + \Delta h_{s} \\
z_{1} - z_{2} &=  \Delta h_{f} + \Delta h_{s} \\
10 [m] &= \Delta h_{f} + \Delta h_{s} \\
10 [m] &=  f\frac{L_{total}}{D} \frac{U^{2}}{2g} + K_{total} \frac{U^{2}}{2g} \\
10 [m] &= \left(f\frac{L_{total}}{D} + K_{total} \right) \frac{U^{2}}{2g}
\end{align*}

A su vez:

$$Re = \dfrac{UD}{\nu}$$

$$ f = funcion\left(Re,\dfrac{\epsilon}{D}\right) $$

El problema debe ser resuelto de forma iterativa.

Procedimiento: \\

1) Suponer un valor de $f_{supuesto}$ \\
2) Calcular $U = \sqrt{\dfrac{10 [m]\cdot 2g}{\left(f_{supuesto}\dfrac{L_{total}}{D} + K_{total} \right)}}$ \\
3) Calcular Número de Reynolds $Re =\dfrac{UD}{\nu}$ \\
4) Recalcular $f$ del Diagrama de Moody (Con $Re$ y $\dfrac{\epsilon}{D}$)

5) Si $f_{supuesto}=f$ entonces OK; sino suponer el valor de $f$ encontrado y reiniciar procedimiento.

In [6]:
def Ecuacion_U2(f_sup,L,g,D,H,K):
  term1 = 2*g*H
  term2 = (f_sup*L/D) + K
  term3 = (term1/term2)
  U = np.sqrt(term3)
  return U


# Datos
epsilon = 0.26 # Rugoisidad absoluta (mm)
D       = 150  # Diámetro (mm)
nu      = 1.10*1e-6 # Viscosidad Cinemática (m2/s)
H       = 10 # Altura de Energía Disponible (m)
L_AB    = 25 # Longitud Tramo A-B (m)
L_BC    = 15 # Longitud Tramo A-B (m)
L_CS    = 50 # Longitud Tramo C-Salida (m)
Ks      = 0.5 # Coef Pérdida Salida del Estanque
Kd      = 0.7 # Coef Pérdida Cambios de Dirección
Ke      = 10 # Coef Pérdida Válvula Esférica
#-----------------------------------------

D       = D/1000 #(m)
epsilon = epsilon/1000 #(m)
L       = L_AB + L_BC + L_CS
K       = Ks + Kd + Ke

e_D = epsilon/D
A   = Area(D)


tol=1e-3
M = np.array([[]])
f_sup = 0.03

i = 0
while True:
  U = Ecuacion_U2(f_sup,L,g,D,H,K)
  #print('U = ',np.round(U,3),' m/s')
  Q = U*A
  #print('Q = ',np.round(Q,3),' m3/s')
  Re = Reynolds(U,D,nu)
  #print('Re = ','%.2E' % Decimal(Re))
  f = f_friccion_NR(e_D,Re)
  #print('f = ','%.6f' % Decimal(f))

  #M.append([f_sup, D, A, Re, e_D, f])
  M = np.append(M, [int(i+1),f_sup, U, Q, Re, f])
  if np.abs((f-f_sup)/f) < tol:
    break
  f_sup = f
  i = i + 1

M = M.reshape(int(len(M)/6),6)
# Convert the NumPy array to a DataFrame
df = pd.DataFrame(M, columns=['Iteración','f_Supuesto','U (m/s)','Q (m^3/s)','Re','f'])
df['Iteración'] = df['Iteración'].map('{:,.0f}'.format)
df['Re'] = df['Re'].map('{:,.2e}'.format)
df['U (m/s)'] = df['U (m/s)'].map('{:.3f}'.format)
df['Q (m^3/s)'] = df['Q (m^3/s)'].map('{:.3f}'.format)

# displaying the DataFrame
display(df)

,Iteración,f_Supuesto,U (m/s),Q (m^3/s),Re,f
0,1,0.030000,2.592,0.046,3.53e+05,0.023124
1,2,0.023124,2.797,0.049,3.81e+05,0.023084
2,3,0.023084,2.799,0.049,3.82e+05,0.023084


### Ejercicio 4
La tubería de pared lisa que se indica en la figura transporta un flujo a razón de 7 m$^{3}$/hr. El fluido transportado posee una densidad de 789 kg/m$^{3}$ y una viscosidad dinámica de 0.0012 kg/ms. Estime las alturas $h_{2}$ y $h_{3}$ despreciando las pérdidas regulares de la tubería corta (entre $h_{2}$ y $h_{1}$) y asuma un $K$ de la pérdida singular debido a la obstrucción de 1.5.

Balance de Energía entre 2 y 1 (Bernoulli):
\begin{align*}
B_{2} &= B_{1} + \Delta h_{s}\\
z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} &= z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} + \Delta h_{s} \\
\frac{P_{2}}{\gamma}  &= \frac{P_{1}}{\gamma} + \Delta h_{s} \\
h_{2} &=  h_{1} + \Delta h_{s} \\
h_{2} &= 1 [m] + K \frac{U^{2}}{2g} \\
h_{2} &= 1 [m] + K \frac{8Q^{2}}{\pi^{2}D^{4}g}
\end{align*}

Una vez conocido $h_{2}$ se puede conocer $h_{3}$ haciendo un balance de energía entre 3 y 2:
\begin{align*}
B_{3} &= B_{3} + \Delta h_{f}\\
z_{3} + \frac{P_{3}}{\gamma} + \frac{U^{2}_{3}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} + \Delta h_{f} \\
\frac{P_{3}}{\gamma}  &= \frac{P_{2}}{\gamma} + \Delta h_{f} \\
h_{3} &= h_{2} + \Delta h_{f} \\
h_{3} &= h_{2} + f\frac{L}{D}\frac{U^{2}}{2g}
\end{align*}

$f$ se determina del diagrama de Moody para una tubería de pared lisa y el Número de Reynolds. Este problema no requiere de iteraaciones y es explícito.

In [8]:
#Datos
Q       = 7 # Caudal (m^3/hr)
rho     = 789 # Densidad (kg/m^3)
mu      = 0.0012 # Viscosidad Dinámica (absoluta) (Pa s)
K       = 1.5 # Coef Pérdida Obstrucción
epsilon = 0 # Rugoisidad absoluta (mm) (Tubería Lisa)
D       = 5 # Diámetro (cm)
h1      = 1 # Altura h1 (m)
L       = 5 # Longitud del tramo entre h3 y h2 (m)

#-------------------------------------

Q = Q/3600 # Q en (m^3/s)
D = D/100 #(m)

e_D = epsilon/D
A   = Area(D)

U = Q/A # Velocidad media (m/s)

h2 = h1 + K*(U**2/(2*g))
print('h2 = ',np.round(h2,3),' m')

Re = Reynolds2(rho,U,D,mu)

f = f_friccion_NR(e_D,Re)

h3 = h2 + f*(L/D)*(U**2/(2*g))
print('h3 = ',np.round(h3,3),' m')

h2 =  1.075  m
h3 =  1.19  m


### Ejercicio 5
Ambas tuberías del sistema en serie indicado en la figura poseen una rugosidad absoluta de 0.1 mm y descargan un caudal de 0.1 m^{3}/s con viscosidad cinemática de $1\cdot 10^{-6}$ m$^{2}$/s. Los diámetros en las tuberías 1 y 2 son 15 y 30 cm, y los largos asociados son de 50 y 160 m, respectivamente. Determine la diferencia en el nivel de la superficie del agua entre los dos estanques, $H$, considerando sólo las pérdidas regulares.

Balance de Energía (Bernoulli):
\begin{align*}
B_{1} &= B_{2} + \Delta h_{f}\\
z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} + \Delta h_{f}\\
z_{1} - z_{2} &= \Delta h_{f} \\
H &= \Delta h_{f_{1}} + \Delta h_{f_{2}} \\
&= f_{1}\frac{L_{1}}{D_{1}} \frac{U^{2}_{1}}{2g} + f_{2}\frac{L_{2}}{D_{2}} \frac{U^{2}_{2}}{2g}
\end{align*}

$f_{1}$ y $f_{2}$ se determinan del diagrama de Moody para una tubería con rugosidad de 0.1 mm y el Número de Reynolds de cada tubería. Este problema no requiere de iteraaciones y es explícito.

In [9]:
#Datos
epsilon = 0.1 # Rugosidad absoluta (mm)
Q       = 0.1 # Caudal (m^3/s)
nu      = 1.0*1e-6 # Viscosidad Cinemática (m^2/s)
D1      = 15 # Diámetro tubería 1 (cm)
D2      = 30 # Diámetro tubería 2 (cm)
L1      = 50 # Longitud tubería 1 (m)
L2      = 160 # Longitud tubería 2 (m)

#-------------------------------------

D1       = D1/100 #(m)
D2       = D2/100 #(m)
epsilon = epsilon/1000 #(m)

e_D1 = epsilon/D1
e_D2 = epsilon/D2

A1   = Area(D1)
A2   = Area(D2)

U1 = Q/A1
U2 = Q/A2

Re1 = Reynolds(U1,D1,nu)
Re2 = Reynolds(U2,D2,nu)

f1 = f_friccion_NR(e_D1,Re1)
f2 = f_friccion_NR(e_D2,Re2)

H = f1*(L1/D1)*(U1**2/(2*g)) + f2*(L2/D2)*(U2**2/(2*g))
print('H = ',np.round(H,3),' m')

H =  10.874  m


### Ejercicio 6
En el sistema indicado en la figura todas las tuberías son nuevas, de rugosidad absoluta 0.26 mm y diámetro 8 cm. El sistema transporta agua a 20°C que se asume con una densidad de 998 kg/m$^{3}$ y una viscosidad dinámica de 0.001 kg/ms. ¿Cuál sería el caudal transportado por el sistema cuando la válvula en la tubería C está cerrada? Desprecie las pérdidas singulares.

Balance de Energía (Bernoulli):

\begin{align*}
B_{1} &= B_{2} + \Delta h_{f}\\
z_{1} + \frac{P_{1}}{\gamma} + \frac{U^{2}_{1}}{2g} &= z_{2} + \frac{P_{2}}{\gamma} + \frac{U^{2}_{2}}{2g} + \Delta h_{f}\\
z_{1} - z_{2} &= \Delta h_{f} \\
25 [m] &= \Delta h_{f_{1}} + \Delta h_{f_{2}} \\
25 [m] &= f\frac{L_{1}}{D} \frac{U^{2}}{2g} + f\frac{L_{2}}{D} \frac{U^{2}}{2g} \\
25 [m] &= \left(L_{1} + L_{2} \right) \frac{fU^{2}}{2gD} \\
25 [m] &= \left(L_{1} + L_{2} \right) \frac{8f}{g\pi^{2}D^{5}} Q^{2}
\end{align*}

A su vez:

$$Re = \dfrac{UD}{\nu}$$

$$ f = funcion\left(Re,\dfrac{\epsilon}{D}\right) $$

El problema debe ser resuelto de forma iterativa.

Procedimiento: \\

1) Suponer un valor de $f_{supuesto}$ \\
2) Calcular $U = \sqrt{\dfrac{25 [m]\cdot 2g\cdot D}{f_{supuesto}\left(L_{1} + L_{2} \right)}}$ \\
3) Calcular Número de Reynolds $Re =\dfrac{\rho UD}{\mu}$ \\
4) Recalcular $f$ del Diagrama de Moody (Con $Re$ y $\dfrac{\epsilon}{D}$)

5) Si $f_{supuesto}=f$ entonces OK; sino suponer el valor de $f$ encontrado y reiniciar procedimiento.

In [10]:
def Ecuacion_U3(f_sup,L,g,D,H):
  term1 = 2*g*H*D
  term2 = f_sup*L
  term3 = (term1/term2)
  U = np.sqrt(term3)
  return U

#Datos
epsilon = 0.26 # Rugosidad absoluta de todas las tuberías (mm)
D       = 8 # Diámetro de todas las tuberías (cm)
rho     = 998 # Densidad (kg/m^3)
mu      = 0.001 # Viscosidad Dinámica (absoluta) (Pa s)
LA      = 100 # Longitud tubería A (m)
LB      = 50 # Longitud tubería B (m)
LC      = 70 # Longitud tubería C (m)
H       = 25 # Altura de Energía Disponible (m)

#-------------------------------------

D       = D/100 #(m)
epsilon = epsilon/1000 #(m)
L       = LA + LB

e_D = epsilon/D
A   = Area(D)

tol=1e-3
M = np.array([[]])
f_sup = 0.03

i = 0
while True:
  U = Ecuacion_U3(f_sup,L,g,D,H)
  #print('U = ',np.round(U,3),' m/s')
  Q = U*A
  #print('Q = ',np.round(Q,3),' m3/s')
  Re = Reynolds2(rho,U,D,mu)
  #print('Re = ','%.2E' % Decimal(Re))
  f = f_friccion_NR(e_D,Re)
  #print('f = ','%.6f' % Decimal(f))

  #M.append([f_sup, D, A, Re, e_D, f])
  M = np.append(M, [int(i+1),f_sup, U, Q, Re, f])
  if np.abs((f-f_sup)/f) < tol:
    break
  f_sup = f
  i = i + 1

M = M.reshape(int(len(M)/6),6)
# Convert the NumPy array to a DataFrame
df = pd.DataFrame(M, columns=['Iteración','f_Supuesto','U (m/s)','Q (m^3/s)','Re','f'])
df['Iteración'] = df['Iteración'].map('{:,.0f}'.format)
df['Re'] = df['Re'].map('{:,.2e}'.format)
df['U (m/s)'] = df['U (m/s)'].map('{:.3f}'.format)
df['Q (m^3/s)'] = df['Q (m^3/s)'].map('{:.3f}'.format)

# displaying the DataFrame
display(df)

,Iteración,f_Supuesto,U (m/s),Q (m^3/s),Re,f
0,1,0.03000,2.953,0.015,2.36e+05,0.027310
1,2,0.02731,3.095,0.016,2.47e+05,0.027286
